## ROS2 Odom vs Setpoint
Plot the difference between odom source and setpoints

In [ ]:
!pip install rosbag2-py rclpy scipy matplotlib numpy==1.25.0 pandas

### 1. Compare 6dof odom from sensor and px4 output

In [ ]:
import rosbag2_py
import rclpy
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

def quaternion_to_euler(quat):
    return R.from_quat([quat.x, quat.y, quat.z, quat.w]).as_euler('xyz', degrees=True)

def get_topic_start_times(bag_path, topics):
    start_times = {}
    reader = rosbag2_py.SequentialReader()
    reader.open(
        rosbag2_py.StorageOptions(uri=bag_path, storage_id='sqlite3'),
        rosbag2_py.ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')
    )
    type_map = {t.name: t.type for t in reader.get_all_topics_and_types()}
    seen = set()
    while reader.has_next() and len(seen) < len(topics):
        topic, _, t_ns = reader.read_next()
        if topic in topics and topic not in seen:
            start_times[topic] = t_ns / 1e9
            seen.add(topic)
    return start_times

def read_bag_data(bag_path, topic_name, min_timestamp, time_offset=0.0, max_time=None, align_to_zero=False):
    reader = rosbag2_py.SequentialReader()
    reader.open(
        rosbag2_py.StorageOptions(uri=bag_path, storage_id='sqlite3'),
        rosbag2_py.ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')
    )
    type_map = {t.name: t.type for t in reader.get_all_topics_and_types()}
    if topic_name not in type_map:
        print(f"Topic {topic_name} not found."); return None, None, None, None

    msg_type = get_message(type_map[topic_name])
    ts, pos, ori, vel = [], [], [], []

    while reader.has_next():
        topic, data, t_ns = reader.read_next()
        if topic != topic_name:
            continue
        t = t_ns / 1e9
        if t < min_timestamp:
            continue
        msg = deserialize_message(data, msg_type)

        # Extract
        try:
            if hasattr(msg, 'pose') and hasattr(msg.pose, 'pose'):
                p = msg.pose.pose.position
                q = msg.pose.pose.orientation
                rpy = quaternion_to_euler(q)
                v = msg.twist.twist.linear
                vel.append([v.x, v.y, v.z])
            elif hasattr(msg, 'position'):
                p = msg.position
                rpy = [0, 0, np.rad2deg(getattr(msg, 'yaw', 0.0))]
                v = getattr(msg, 'velocity', type('', (), {'x':0,'y':0,'z':0})())
                vel.append([v.x, v.y, v.z])
            else:
                continue
            ts.append(t)
            pos.append([p.x, p.y, p.z])
            ori.append(rpy)
        except Exception as e:
            print(f"Skipping corrupt message: {e}")
            continue

    if not ts: return None, None, None, None
    ts = np.array(ts) + time_offset
    if align_to_zero: ts -= ts[0]
    pos = np.array(pos); ori = np.array(ori); vel = np.array(vel)
    if max_time:
        idx = np.where(ts - ts[0] <= max_time)
        return ts[idx], pos[idx], ori[idx], vel[idx]
    return ts, pos, ori, vel

def plot_data(bag_path, topics, min_timestamp, time_offsets=None, max_time=None, align_to_zero=True):
    fig, axs = plt.subplots(3, 3, figsize=(18, 14))
    fig.suptitle("Aligned Odometry and Setpoint Data", fontsize=16)
    colors = ['blue', 'red', 'green', 'orange']

    for i, topic in enumerate(topics):
        offset = time_offsets.get(topic, 0.0) if time_offsets else 0.0
        ts, pos, ori, vel = read_bag_data(bag_path, topic, min_timestamp, offset, max_time, align_to_zero)
        if ts is None: continue
        c = colors[i % len(colors)]
        label = topic.split('/')[-1]

        axs[0,0].plot(ts, pos[:,0], label=label, color=c)
        axs[0,1].plot(ts, pos[:,1], label=label, color=c)
        axs[0,2].plot(ts, pos[:,2], label=label, color=c)
        axs[1,0].plot(ts, vel[:,0], label=label, color=c)
        axs[1,1].plot(ts, vel[:,1], label=label, color=c)
        axs[1,2].plot(ts, vel[:,2], label=label, color=c)
        axs[2,0].plot(ts, ori[:,0], label=label, color=c)
        axs[2,1].plot(ts, ori[:,1], label=label, color=c)
        axs[2,2].plot(ts, ori[:,2], label=label, color=c)

    titles = [['X (m)', 'Y (m)', 'Z (m)'],
              ['Vx (m/s)', 'Vy (m/s)', 'Vz (m/s)'],
              ['Roll (°)', 'Pitch (°)', 'Yaw (°)']]
    for i in range(3):
        for j in range(3):
            axs[i,j].set_title(titles[i][j])
            axs[i,j].set_xlabel("Time (s)")
            axs[i,j].grid(True)
            if i == 0 and j == 0:
                axs[i,j].legend()
    plt.tight_layout()
    plt.show()

    # 3D Trajectory
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    for i, topic in enumerate(topics):
        offset = time_offsets.get(topic, 0.0) if time_offsets else 0.0
        ts, pos, *_ = read_bag_data(bag_path, topic, min_timestamp, offset, max_time, align_to_zero)
        if ts is None: continue
        label = topic.split('/')[-1]
        c = colors[i % len(colors)]
        ax.plot(pos[:,0], pos[:,1], pos[:,2], label=label, color=c)
        ax.scatter(pos[0,0], pos[0,1], pos[0,2], marker='o', s=100, color=c)
        ax.scatter(pos[-1,0], pos[-1,1], pos[-1,2], marker='^', s=100, color=c)
    ax.set_title("3D Trajectory"); ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
    ax.legend(); ax.grid(True); plt.show()

# === USAGE ===
bag_path = "../bags/good1"
topics = ["/mavros/setpoint_raw/local", "/mavros/odometry/in", "/mavros/odometry/out"]
start_times = get_topic_start_times(bag_path, topics)
min_timestamp = max(start_times.values())  # Only plot from latest start

plot_data(bag_path, topics, min_timestamp, time_offsets=None, max_time=50, align_to_zero=True)

### 2. Compare range sensor readings (raw and scaled with roll pitch angle) with height from odom 

In [ ]:
import rosbag2_py
import rclpy
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R

def quaternion_to_euler(quat):
    return R.from_quat([quat.x, quat.y, quat.z, quat.w]).as_euler('xyz', degrees=True)

def get_topic_start_times(bag_path, topics):
    """
    Reads the bag file to find the first timestamp for each specified topic.
    """
    start_times = {}
    reader = rosbag2_py.SequentialReader()
    reader.open(
        rosbag2_py.StorageOptions(uri=bag_path, storage_id='sqlite3'),
        rosbag2_py.ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')
    )
    type_map = {t.name: t.type for t in reader.get_all_topics_and_types()}
    seen = set()
    
    # Set filters to read only the first message of the specified topics
    reader.set_filter(
        rosbag2_py.StorageFilter(topics=topics)
    )

    while reader.has_next() and len(seen) < len(topics):
        topic, _, t_ns = reader.read_next()
        if topic in topics and topic not in seen:
            start_times[topic] = t_ns / 1e9
            seen.add(topic)
    return start_times

def read_odometry_data(bag_path, topic_name, min_common_timestamp=0.0, max_time_duration=None, align_to_zero=False):
    """
    Reads odometry data from a rosbag2 file, filters by a minimum common timestamp,
    and optionally aligns timestamps to start from zero relative to the first filtered timestamp.
    """
    reader = rosbag2_py.SequentialReader()
    reader.open(
        rosbag2_py.StorageOptions(uri=bag_path, storage_id='sqlite3'),
        rosbag2_py.ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')
    )
    type_map = {t.name: t.type for t in reader.get_all_topics_and_types()}
    if topic_name not in type_map:
        print(f"Topic {topic_name} not found."); return None, None, None

    msg_type = get_message(type_map[topic_name])
    ts, pos, ori = [], [], []

    reader.set_filter(
        rosbag2_py.StorageFilter(topics=[topic_name])
    )

    while reader.has_next():
        topic, data, t_ns = reader.read_next()
        
        t = t_ns / 1e9
        if t < min_common_timestamp: # Filter data before the common start time
            continue
        
        msg = deserialize_message(data, msg_type)

        try:
            if hasattr(msg, 'pose') and hasattr(msg.pose, 'pose'):
                p = msg.pose.pose.position
                q = msg.pose.pose.orientation
                rpy = quaternion_to_euler(q)
            else:
                continue
            ts.append(t)
            pos.append([p.x, p.y, p.z])
            ori.append(rpy)
        except Exception as e:
            print(f"Skipping corrupt odometry message: {e}")
            continue

    if not ts: return None, None, None
    
    ts = np.array(ts)
    if align_to_zero: 
        ts -= ts[0] # Make the first timestamp 0
    
    pos = np.array(pos)
    ori = np.array(ori)
    
    if max_time_duration:
        # If align_to_zero is true, ts[0] will be 0, otherwise it's the actual first timestamp
        idx = np.where(ts <= max_time_duration if align_to_zero else (ts - ts[0] <= max_time_duration))
        return ts[idx], pos[idx], ori[idx]
    
    return ts, pos, ori

def read_range_data(bag_path, topic_name, min_common_timestamp=0.0, max_time_duration=None, align_to_zero=False):
    """
    Reads range sensor data from a rosbag2 file, filters by a minimum common timestamp,
    and optionally aligns timestamps to start from zero relative to the first filtered timestamp.
    """
    reader = rosbag2_py.SequentialReader()
    reader.open(
        rosbag2_py.StorageOptions(uri=bag_path, storage_id='sqlite3'),
        rosbag2_py.ConverterOptions(input_serialization_format='cdr', output_serialization_format='cdr')
    )
    type_map = {t.name: t.type for t in reader.get_all_topics_and_types()}
    if topic_name not in type_map:
        print(f"Topic {topic_name} not found."); return None, None

    msg_type = get_message(type_map[topic_name])
    ts, ranges = [], []

    reader.set_filter(
        rosbag2_py.StorageFilter(topics=[topic_name])
    )

    while reader.has_next():
        topic, data, t_ns = reader.read_next()
        
        t = t_ns / 1e9
        if t < min_common_timestamp: # Filter data before the common start time
            continue
        
        msg = deserialize_message(data, msg_type)

        try:
            range_val = msg.range
            if msg.min_range <= range_val <= msg.max_range:
                ts.append(t)
                ranges.append(range_val)
        except Exception as e:
            print(f"Skipping corrupt range message: {e}")
            continue

    if not ts: return None, None
    
    ts = np.array(ts)
    if align_to_zero: 
        ts -= ts[0] # Make the first timestamp 0
    
    ranges = np.array(ranges)
    
    if max_time_duration:
        idx = np.where(ts <= max_time_duration if align_to_zero else (ts - ts[0] <= max_time_duration))
        return ts[idx], ranges[idx]
    
    return ts, ranges

def adjust_range_for_attitude(raw_range, roll_deg, pitch_deg):
    roll_rad = np.deg2rad(roll_deg)
    pitch_rad = np.deg2rad(pitch_deg)
    cos_roll = np.cos(roll_rad)
    cos_pitch = np.cos(pitch_rad)
    attitude_factor = cos_roll * cos_pitch
    attitude_factor = np.maximum(attitude_factor, 0.1) # Prevent division by zero or very small numbers
    adjusted_range = raw_range * attitude_factor
    return adjusted_range

def plot_range_height_comparison(bag_path, odom_topic, range_topic, max_time_duration=None, align_to_zero=True):
    """
    Plots comparison of range sensor data (raw + attitude adjusted) and odometry height,
    aligned by the first range message timestamp. Also prints relevant stats.
    """
    # Read full range data first
    range_ts, ranges = read_range_data(
        bag_path, range_topic, min_common_timestamp=0.0, max_time_duration=None, align_to_zero=False
    )
    if range_ts is None or len(range_ts) == 0:
        print("No range data found.")
        return

    # Use the first valid range timestamp as alignment reference
    min_range_time = range_ts[0]

    # Read odometry data filtered to start at the first range timestamp
    odom_ts_full, odom_pos_full, odom_ori_full = read_odometry_data(
        bag_path, odom_topic, min_common_timestamp=min_range_time, max_time_duration=None, align_to_zero=False
    )
    if odom_ts_full is None or len(odom_ts_full) == 0:
        print("No odometry data found.")
        return

    # Trim odometry to match range start
    odom_mask = odom_ts_full >= min_range_time
    odom_ts = odom_ts_full[odom_mask]
    odom_pos = odom_pos_full[odom_mask]
    odom_ori = odom_ori_full[odom_mask]

    # Optionally trim based on max_time_duration
    if max_time_duration is not None:
        range_mask = (range_ts - min_range_time) <= max_time_duration
        odom_mask = (odom_ts - min_range_time) <= max_time_duration
        range_ts = range_ts[range_mask]
        ranges = ranges[range_mask]
        odom_ts = odom_ts[odom_mask]
        odom_pos = odom_pos[odom_mask]
        odom_ori = odom_ori[odom_mask]

    # Align time to zero if needed
    if align_to_zero:
        range_ts -= min_range_time
        odom_ts -= min_range_time

    # Interpolate roll and pitch to range timestamps
    roll_interp = np.interp(range_ts, odom_ts, odom_ori[:, 0])
    pitch_interp = np.interp(range_ts, odom_ts, odom_ori[:, 1])
    height_interp = np.interp(range_ts, odom_ts, odom_pos[:, 2])

    # Adjust range for attitude
    adjusted_ranges = adjust_range_for_attitude(ranges, roll_interp, pitch_interp)

    # === Plotting ===
    fig, ax = plt.subplots(figsize=(12, 8))

    ax.plot(range_ts, ranges, label='Raw Range', color='red', linewidth=2)
    ax.plot(range_ts, adjusted_ranges, label='Attitude Adjusted Range', color='blue', linewidth=2)
    ax.plot(range_ts, height_interp, label='Odometry Height (Z)', color='green', linewidth=2)

    ax.set_title('Range Sensor vs Odometry Height Comparison', fontsize=16)
    ax.set_xlabel('Time (s)', fontsize=14)
    ax.set_ylabel('Height/Distance (m)', fontsize=14)
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=12)
    plt.tight_layout()
    plt.show()

    # === Print stats ===
    print("\n=== Summary Statistics ===")
    print(f"Mean Raw Range:           {np.mean(ranges):.3f} m")
    print(f"Mean Adjusted Range:      {np.mean(adjusted_ranges):.3f} m")
    print(f"Mean Odometry Height (Z): {np.mean(height_interp):.3f} m")
    print(f"Max Roll Angle:           {np.max(np.abs(roll_interp)):.2f}°")
    print(f"Max Pitch Angle:          {np.max(np.abs(pitch_interp)):.2f}°")


# === USAGE ===
bag_path = "../bags/test1"
odom_topic = "/mavros/odometry/out"
range_topic = "/mavros/hps167_pub"

plot_range_height_comparison(bag_path, odom_topic, range_topic, max_time_duration=50, align_to_zero=True)

### 3. Generate Polynomial Traj output and plot it

In [ ]:
import rclpy
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message
import rosbag2_py
import numpy as np
import math
import csv

from uosm_uav_interface.msg import PolynomialTraj

def get_rosbag_options(path, serialization_format='cdr'):
    """
    Helper function to create rosbag2_py.StorageOptions and rosbag2_py.ConverterOptions.
    """
    storage_options = rosbag2_py.StorageOptions(uri=path, storage_id='sqlite3')
    converter_options = rosbag2_py.ConverterOptions(
        input_serialization_format=serialization_format,
        output_serialization_format=serialization_format
    )
    return storage_options, converter_options

class TrajectoryProcessor:
    def __init__(self, time_forward_=0.1):
        self.time_forward_ = time_forward_
        self.last_yaw_ = 0.0 # Initial value for last_yaw_

    def get_position(self, t: float, coef_x_piece: np.ndarray, coef_y_piece: np.ndarray, coef_z_piece: np.ndarray, order: int) -> np.ndarray:
        """
        Evaluates the position for a given time t within a polynomial piece.
        The coefficients are in descending order of power (e.g., C5*t^5 + C4*t^4 + ... + C0).
        """
        pos = np.zeros(3)
        for i in range(order + 1):
            power = order - i
            pos[0] += coef_x_piece[i] * (t ** power)
            pos[1] += coef_y_piece[i] * (t ** power)
            pos[2] += coef_z_piece[i] * (t ** power)
        return pos

    def calculate_yaw_and_pos(self, msg: PolynomialTraj, time_step=0.01):
        """
        Processes a single PolynomialTraj message and returns time, yaw, x, y, z points.
        time_step determines the granularity of the generated data points.
        """
        if msg.order != 5:
            print(f"Skipping trajectory with order {msg.order}. Only order 5 is supported.")
            return [], [], [], [], []

        piece_nums = len(msg.duration)
        durations = msg.duration

        expected_coef_len = piece_nums * (msg.order + 1)
        if len(msg.coef_x) != expected_coef_len or \
           len(msg.coef_y) != expected_coef_len or \
           len(msg.coef_z) != expected_coef_len:
            print(f"Coefficient array length mismatch. Expected {expected_coef_len}, got x:{len(msg.coef_x)}, y:{len(msg.coef_y)}, z:{len(msg.coef_z)}. Skipping message.")
            return [], [], [], [], []

        coef_x_mat = np.array(msg.coef_x).reshape(piece_nums, msg.order + 1)
        coef_y_mat = np.array(msg.coef_y).reshape(piece_nums, msg.order + 1)
        coef_z_mat = np.array(msg.coef_z).reshape(piece_nums, msg.order + 1)

        total_duration = sum(durations)
        if total_duration == 0: # Avoid division by zero if duration is somehow empty
            return [], [], [], [], []

        # Generate time points within this specific trajectory
        time_points_traj = np.arange(0, total_duration, time_step)
        if total_duration not in time_points_traj: # Ensure the end point is included
            time_points_traj = np.append(time_points_traj, total_duration)

        yaw_points = []
        x_points = []
        y_points = []
        z_points = []
        absolute_times = [] # To store the absolute ROS time stamp

        traj_start_sec = msg.start_time.sec
        traj_start_nanosec = msg.start_time.nanosec
        
        current_time_offset = 0.0
        current_piece_idx = 0

        for t_cur_relative in time_points_traj:
            # Adjust current_piece_idx and current_time_offset for the current t_cur_relative
            while current_piece_idx < piece_nums - 1 and t_cur_relative >= (current_time_offset + durations[current_piece_idx]):
                current_time_offset += durations[current_piece_idx]
                current_piece_idx += 1
            
            t_in_piece = t_cur_relative - current_time_offset

            # Calculate position at t_cur_relative
            pos = self.get_position(t_in_piece, coef_x_mat[current_piece_idx], coef_y_mat[current_piece_idx], coef_z_mat[current_piece_idx], msg.order)
            x_points.append(pos[0])
            y_points.append(pos[1])
            z_points.append(pos[2])

            # Calculate position for yaw look-ahead
            t_look_ahead_relative = t_cur_relative + self.time_forward_
            
            if t_look_ahead_relative > total_duration:
                # If look-ahead time goes beyond the trajectory, use the end point of the last piece
                pos_look_ahead = self.get_position(
                    durations[piece_nums - 1], # time within the last piece
                    coef_x_mat[piece_nums - 1],
                    coef_y_mat[piece_nums - 1],
                    coef_z_mat[piece_nums - 1],
                    msg.order
                )
            else:
                # Find the correct piece for the look-ahead time
                look_ahead_piece_idx = 0
                look_ahead_time_offset = 0.0
                for i in range(piece_nums):
                    if t_look_ahead_relative < (look_ahead_time_offset + durations[i]):
                        look_ahead_piece_idx = i
                        break
                    look_ahead_time_offset += durations[i]
                    if i == piece_nums - 1: # Fallback if it's the very end of the last piece
                        look_ahead_piece_idx = i

                t_in_look_ahead_piece = t_look_ahead_relative - look_ahead_time_offset
                pos_look_ahead = self.get_position(
                    t_in_look_ahead_piece,
                    coef_x_mat[look_ahead_piece_idx],
                    coef_y_mat[look_ahead_piece_idx],
                    coef_z_mat[look_ahead_piece_idx],
                    msg.order
                )

            # Calculate direction vector
            dir_vec = pos_look_ahead - pos

            # Calculate yaw
            if np.linalg.norm(dir_vec[:2]) > 0.1: # Only consider XY plane for yaw
                yaw = math.atan2(dir_vec[1], dir_vec[0])
                self.last_yaw_ = yaw # Update last_yaw_ for future use if movement is significant
            else:
                yaw = self.last_yaw_ # Hold last yaw if movement is insignificant

            # Normalize yaw to [-pi, pi]
            if yaw > math.pi:
                yaw -= 2.0 * math.pi
            if yaw < -math.pi:
                yaw += 2.0 * math.pi

            yaw_points.append(yaw)

            # Calculate absolute time for this data point
            # Assuming msg.start_time is the start of this specific trajectory instance
            absolute_time_ns = traj_start_sec * 1_000_000_000 + traj_start_nanosec + int(t_cur_relative * 1_000_000_000)
            absolute_times.append(absolute_time_ns / 1_000_000_000.0) # Convert to seconds

        return absolute_times, yaw_points, x_points, y_points, z_points


def process_bag_to_csv(bag_file_path, output_csv_path, topic_name="/ego_planner/poly_traj", time_forward=0.1, time_step=0.01):
    """
    Reads a ROS 2 bag file, processes PolynomialTraj messages, and writes data to a CSV.
    """
    storage_options, converter_options = get_rosbag_options(bag_file_path)
    reader = rosbag2_py.SequentialReader()
    reader.open(storage_options, converter_options)

    # Filter messages to only the desired topic
    topic_types = reader.get_all_topics_and_types()
    type_map = {topic_types[i].name: topic_types[i].type for i in range(len(topic_types))}
    
    # Check if the topic exists in the bag
    if topic_name not in type_map:
        print(f"Error: Topic '{topic_name}' not found in the bag file.")
        return
    
    processor = TrajectoryProcessor(time_forward_=time_forward)

    all_data = [] # To store all rows before writing to CSV

    # CSV header
    header = ['timestamp', 'yaw', 'pos_x', 'pos_y', 'pos_z']
    all_data.append(header)

    try:
        message_type = get_message(type_map[topic_name])
        message_count = 0
        
        # Use has_next() and read_next() instead of read_messages()
        while reader.has_next():
            (topic, data, timestamp) = reader.read_next()
            if topic == topic_name:
                msg = deserialize_message(data, message_type)
                
                # Process the message to get time, yaw, x, y, z points for this trajectory
                times, yaws, xs, ys, zs = processor.calculate_yaw_and_pos(msg, time_step)

                for j in range(len(times)):
                    all_data.append([times[j], yaws[j], xs[j], ys[j], zs[j]])

                message_count += 1
                print(f"Processed trajectory message {message_count} at ROS timestamp {timestamp / 1e9:.6f} s.")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()


    # Write all collected data to CSV
    with open(output_csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(all_data)

    print(f"Data extracted and saved to '{output_csv_path}'")

if __name__ == '__main__':

    bag_path = '../bags/good1/test1_0.db3'
    csv_output_path = 'trajectory_yaw_data.csv'

    # Adjust time_forward and time_step if needed
    process_bag_to_csv(bag_path, csv_output_path, time_forward=1.0, time_step=0.01)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

def plot_trajectory_data(csv_path='trajectory_yaw_data.csv'):
    """
    Plots trajectory data from the CSV file generated by the bag processor.
    Creates multiple plots: 3D trajectory, 2D trajectory, yaw over time, and position components over time.
    """
    try:
        # Read the CSV file
        df = pd.read_csv(csv_path)
        
        # Extract data
        timestamps = df['timestamp'].values
        yaw = df['yaw'].values
        pos_x = df['pos_x'].values
        pos_y = df['pos_y'].values
        pos_z = df['pos_z'].values
        
        # Convert timestamps to relative time (starting from 0)
        time_relative = timestamps - timestamps[0]
        
        # Create a figure with multiple subplots
        fig = plt.figure(figsize=(16, 12))
        
        # 1. 3D Trajectory Plot
        ax1 = fig.add_subplot(2, 3, 1, projection='3d')
        
        # Create a colormap based on time for the trajectory
        colors = plt.cm.viridis(np.linspace(0, 1, len(pos_x)))
        
        # Plot 3D trajectory
        ax1.scatter(pos_x, pos_y, pos_z, c=colors, s=1, alpha=0.7)
        ax1.plot(pos_x, pos_y, pos_z, 'b-', alpha=0.3, linewidth=0.5)
        
        # Mark start and end points
        ax1.scatter(pos_x[0], pos_y[0], pos_z[0], color='green', s=100, marker='o', label='Start')
        ax1.scatter(pos_x[-1], pos_y[-1], pos_z[-1], color='red', s=100, marker='s', label='End')
        
        ax1.set_xlabel('X Position (m)')
        ax1.set_ylabel('Y Position (m)')
        ax1.set_zlabel('Z Position (m)')
        ax1.set_title('3D Trajectory')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 2. 2D Trajectory Plot (X-Y plane)
        ax2 = fig.add_subplot(2, 3, 2)
        
        # Plot 2D trajectory with color gradient
        scatter = ax2.scatter(pos_x, pos_y, c=time_relative, cmap='viridis', s=2, alpha=0.7)
        ax2.plot(pos_x, pos_y, 'b-', alpha=0.3, linewidth=0.5)
        
        # Mark start and end points
        ax2.scatter(pos_x[0], pos_y[0], color='green', s=100, marker='o', label='Start')
        ax2.scatter(pos_x[-1], pos_y[-1], color='red', s=100, marker='s', label='End')
        
        # Add yaw direction arrows at regular intervals
        arrow_interval = max(1, len(pos_x) // 20)  # Show about 20 arrows
        for i in range(0, len(pos_x), arrow_interval):
            dx = 0.5 * np.cos(yaw[i])  # Arrow length
            dy = 0.5 * np.sin(yaw[i])
            ax2.arrow(pos_x[i], pos_y[i], dx, dy, 
                     head_width=0.1, head_length=0.1, 
                     fc='red', ec='red', alpha=0.7)
        
        ax2.set_xlabel('X Position (m)')
        ax2.set_ylabel('Y Position (m)')
        ax2.set_title('2D Trajectory (X-Y plane) with Yaw Arrows')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        ax2.set_aspect('equal')
        
        # Add colorbar for time
        cbar = plt.colorbar(scatter, ax=ax2)
        cbar.set_label('Time (s)')
        
        # 3. Yaw over Time
        ax3 = fig.add_subplot(2, 3, 3)
        ax3.plot(time_relative, np.degrees(yaw), 'r-', linewidth=2)
        ax3.set_xlabel('Time (s)')
        ax3.set_ylabel('Yaw (degrees)')
        ax3.set_title('Yaw Angle Over Time')
        ax3.grid(True, alpha=0.3)
        
        # 4. Position Components over Time
        ax4 = fig.add_subplot(2, 3, 4)
        ax4.plot(time_relative, pos_x, 'r-', label='X', linewidth=2)
        ax4.plot(time_relative, pos_y, 'g-', label='Y', linewidth=2)
        ax4.plot(time_relative, pos_z, 'b-', label='Z', linewidth=2)
        ax4.set_xlabel('Time (s)')
        ax4.set_ylabel('Position (m)')
        ax4.set_title('Position Components Over Time')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
        
        # 5. Velocity Profile (approximate)
        ax5 = fig.add_subplot(2, 3, 5)
        if len(time_relative) > 1:
            dt = np.diff(time_relative)
            dx = np.diff(pos_x)
            dy = np.diff(pos_y)
            dz = np.diff(pos_z)
            
            # Avoid division by zero
            dt[dt == 0] = 1e-6
            
            vel_x = dx / dt
            vel_y = dy / dt
            vel_z = dz / dt
            vel_mag = np.sqrt(vel_x**2 + vel_y**2 + vel_z**2)
            
            ax5.plot(time_relative[:-1], vel_x, 'r-', label='Vx', alpha=0.7)
            ax5.plot(time_relative[:-1], vel_y, 'g-', label='Vy', alpha=0.7)
            ax5.plot(time_relative[:-1], vel_z, 'b-', label='Vz', alpha=0.7)
            ax5.plot(time_relative[:-1], vel_mag, 'k-', label='|V|', linewidth=2)
            
        ax5.set_xlabel('Time (s)')
        ax5.set_ylabel('Velocity (m/s)')
        ax5.set_title('Velocity Profile (Approximate)')
        ax5.legend()
        ax5.grid(True, alpha=0.3)
        
        # 6. Yaw Rate (approximate)
        ax6 = fig.add_subplot(2, 3, 6)
        if len(time_relative) > 1:
            dt = np.diff(time_relative)
            dyaw = np.diff(yaw)
            
            # Handle yaw wrap-around
            dyaw[dyaw > np.pi] -= 2 * np.pi
            dyaw[dyaw < -np.pi] += 2 * np.pi
            
            # Avoid division by zero
            dt[dt == 0] = 1e-6
            
            yaw_rate = dyaw / dt
            
            ax6.plot(time_relative[:-1], np.degrees(yaw_rate), 'purple', linewidth=2)
            
        ax6.set_xlabel('Time (s)')
        ax6.set_ylabel('Yaw Rate (deg/s)')
        ax6.set_title('Yaw Rate Over Time')
        ax6.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Print some statistics
        print(f"\n=== Trajectory Statistics ===")
        print(f"Total trajectory points: {len(timestamps)}")
        print(f"Total duration: {time_relative[-1]:.2f} seconds")
        print(f"Position range:")
        print(f"  X: [{pos_x.min():.2f}, {pos_x.max():.2f}] m")
        print(f"  Y: [{pos_y.min():.2f}, {pos_y.max():.2f}] m")
        print(f"  Z: [{pos_z.min():.2f}, {pos_z.max():.2f}] m")
        print(f"Yaw range: [{np.degrees(yaw.min()):.1f}, {np.degrees(yaw.max()):.1f}] degrees")
        
        if len(time_relative) > 1:
            total_distance = np.sum(np.sqrt(np.diff(pos_x)**2 + np.diff(pos_y)**2 + np.diff(pos_z)**2))
            avg_speed = total_distance / time_relative[-1]
            print(f"Total distance: {total_distance:.2f} m")
            print(f"Average speed: {avg_speed:.2f} m/s")
        
    except FileNotFoundError:
        print(f"Error: CSV file '{csv_path}' not found. Make sure to run the bag processor first.")
    except Exception as e:
        print(f"Error plotting data: {e}")
        import traceback
        traceback.print_exc()

if __name__ == '__main__':
    # Plot the trajectory data
    print("Plotting detailed trajectory analysis...")
    plot_trajectory_data('trajectory_yaw_data.csv')